<a href="https://colab.research.google.com/github/MaliheDahmardeh/Olympic-History/blob/main/Olympic_History.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [154]:
import numpy as np
import pandas as pd
import os

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sb
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [155]:
!ls

athlete_events.csv  noc_regions.csv  sample_data


In [156]:
df_events = pd.read_csv('athlete_events.csv')

In [157]:
df_noc = pd.read_csv('noc_regions.csv')

In [158]:
df = pd.merge(df_events,df_noc,on='NOC',how='left')
df

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN,Poland,NaN
271112,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN,Poland,NaN
271113,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN,Poland,NaN
271114,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN,Poland,NaN


In [159]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271116 entries, 0 to 271115
Data columns (total 17 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   39783 non-null   object 
 15  region  270746 non-null  object 
 16  notes   5039 non-null    object 
dtypes: float64(3), int64(2), object(12)
memory usage: 37.2+ MB


#Data Cleaning

In [160]:
df.isnull().sum()

ID             0
Name           0
Sex            0
Age         9474
Height     60171
Weight     62875
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     231333
region       370
notes     266077
dtype: int64

In [161]:
#drop columns 'notes' and 'ID' because they are not important and column 'notes' has lots of null values
df.drop(columns=['notes','ID'],inplace=True)

In [162]:
# check and drop duplicated rows
df.duplicated().sum()

1385

In [163]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

In [164]:
df.isnull().sum()

Name           0
Sex            0
Age         9315
Height     58814
Weight     61527
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     229959
region       370
dtype: int64

Fill  null value for Age, Height and Weight with mean of them

In [165]:
#Mean of height
mean_height=df["Height"].mean()
rmh=round(mean_height)
rmh

175

In [166]:
df["Height"]=df["Height"].fillna(rmh)
df["Height"]

0         180.0
1         170.0
2         175.0
3         175.0
4         185.0
          ...  
271111    179.0
271112    176.0
271113    176.0
271114    185.0
271115    185.0
Name: Height, Length: 269731, dtype: float64

In [167]:
#Mean of weight
mean_weight=df["Weight"].mean()
rmw=round(mean_weight)
rmw

71

In [168]:
df["Weight"]=df["Weight"].fillna(rmw)
df["Weight"]

0         80.0
1         60.0
2         71.0
3         71.0
4         82.0
          ... 
271111    89.0
271112    59.0
271113    59.0
271114    96.0
271115    96.0
Name: Weight, Length: 269731, dtype: float64

In [169]:
#Mean of Age
mean_age=df["Age"].mean()
rma=round(mean_age)
rma

25

In [170]:
df["Age"]=df["Age"].fillna(rma)
df["Age"]

0         24.0
1         23.0
2         24.0
3         34.0
4         21.0
          ... 
271111    29.0
271112    27.0
271113    27.0
271114    30.0
271115    34.0
Name: Age, Length: 269731, dtype: float64

In [171]:
#Changing float type data to integer
df["Age"]=df["Age"].astype(int)
df["Height"]=df["Height"].astype(int)
df["Weight"]=df["Weight"].astype(int)

In [172]:
df.columns

Index(['Name', 'Sex', 'Age', 'Height', 'Weight', 'Team', 'NOC', 'Games',
       'Year', 'Season', 'City', 'Sport', 'Event', 'Medal', 'region'],
      dtype='object')

In [173]:
#drop Medal null values
drop_rows = df.dropna( how='any',subset=['Medal'], inplace=True)

In [174]:
df.isnull().sum()

Name      0
Sex       0
Age       0
Height    0
Weight    0
Team      0
NOC       0
Games     0
Year      0
Season    0
City      0
Sport     0
Event     0
Medal     0
region    9
dtype: int64

In [175]:
#fill remaining null values in region with 'unknown'
df["region"]=df["region"].fillna('unknown')
df["region"]

3         Denmark
37        Finland
38        Finland
40        Finland
41        Finland
           ...   
271078     Russia
271080     Russia
271082     Poland
271102     Russia
271103     Russia
Name: region, Length: 39772, dtype: object

In [176]:
#view of "unknown" region
region_unknown=df.loc[(df["region"]=="unknown")]
region_unknown

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region
67723,Feng Tian Wei,F,21,163,55,Singapore,SGP,2008 Summer,2008,Summer,Beijing,Table Tennis,Table Tennis Women's Team,Silver,unknown
67724,Feng Tian Wei,F,25,163,55,Singapore,SGP,2012 Summer,2012,Summer,London,Table Tennis,Table Tennis Women's Singles,Bronze,unknown
67725,Feng Tian Wei,F,25,163,55,Singapore,SGP,2012 Summer,2012,Summer,London,Table Tennis,Table Tennis Women's Team,Bronze,unknown
138095,Li Jia Wei,F,26,170,60,Singapore,SGP,2008 Summer,2008,Summer,Beijing,Table Tennis,Table Tennis Women's Team,Silver,unknown
138096,Li Jia Wei,F,30,170,60,Singapore,SGP,2012 Summer,2012,Summer,London,Table Tennis,Table Tennis Women's Team,Bronze,unknown
213955,Joseph Isaac Schooling,M,21,184,74,Singapore,SGP,2016 Summer,2016,Summer,Rio de Janeiro,Swimming,Swimming Men's 100 metres Butterfly,Gold,unknown
235908,"Howe Liang ""Tiger"" Tan",M,27,160,69,Singapore,SGP,1960 Summer,1960,Summer,Roma,Weightlifting,Weightlifting Men's Lightweight,Silver,unknown
256622,Wang Jue Gu,F,28,155,63,Singapore,SGP,2008 Summer,2008,Summer,Beijing,Table Tennis,Table Tennis Women's Team,Silver,unknown
256624,Wang Jue Gu,F,32,155,63,Singapore,SGP,2012 Summer,2012,Summer,London,Table Tennis,Table Tennis Women's Team,Bronze,unknown


In [177]:
#because in the Noc_region data frame NOC for Singapore is SIN and in the athlete_events data frame it is SGP we missed some data so we replace "unknown" with "singapore"
df["region"]=df["region"].replace('unknown','Singapore')
region_singapore=df.loc[(df["region"]=="Singapore")]
region_singapore

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region
67723,Feng Tian Wei,F,21,163,55,Singapore,SGP,2008 Summer,2008,Summer,Beijing,Table Tennis,Table Tennis Women's Team,Silver,Singapore
67724,Feng Tian Wei,F,25,163,55,Singapore,SGP,2012 Summer,2012,Summer,London,Table Tennis,Table Tennis Women's Singles,Bronze,Singapore
67725,Feng Tian Wei,F,25,163,55,Singapore,SGP,2012 Summer,2012,Summer,London,Table Tennis,Table Tennis Women's Team,Bronze,Singapore
138095,Li Jia Wei,F,26,170,60,Singapore,SGP,2008 Summer,2008,Summer,Beijing,Table Tennis,Table Tennis Women's Team,Silver,Singapore
138096,Li Jia Wei,F,30,170,60,Singapore,SGP,2012 Summer,2012,Summer,London,Table Tennis,Table Tennis Women's Team,Bronze,Singapore
213955,Joseph Isaac Schooling,M,21,184,74,Singapore,SGP,2016 Summer,2016,Summer,Rio de Janeiro,Swimming,Swimming Men's 100 metres Butterfly,Gold,Singapore
235908,"Howe Liang ""Tiger"" Tan",M,27,160,69,Singapore,SGP,1960 Summer,1960,Summer,Roma,Weightlifting,Weightlifting Men's Lightweight,Silver,Singapore
256622,Wang Jue Gu,F,28,155,63,Singapore,SGP,2008 Summer,2008,Summer,Beijing,Table Tennis,Table Tennis Women's Team,Silver,Singapore
256624,Wang Jue Gu,F,32,155,63,Singapore,SGP,2012 Summer,2012,Summer,London,Table Tennis,Table Tennis Women's Team,Bronze,Singapore


In [178]:
df.isnull().sum()

Name      0
Sex       0
Age       0
Height    0
Weight    0
Team      0
NOC       0
Games     0
Year      0
Season    0
City      0
Sport     0
Event     0
Medal     0
region    0
dtype: int64

#Data Exploration

In [179]:
df.describe()

,Age,Height,Weight,Year
count,39772.000000,39772.000000,39772.000000,39772.000000
mean,25.906215,176.995474,73.119054,1973.964296
std,5.857790,9.686490,13.191658,33.805168
min,10.000000,136.000000,28.000000,1896.000000
25%,22.000000,172.000000,66.000000,1952.000000
50%,25.000000,175.000000,71.000000,1984.000000
75%,29.000000,183.000000,79.000000,2002.000000
max,73.000000,223.000000,182.000000,2016.000000


In [180]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39772 entries, 3 to 271103
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    39772 non-null  object
 1   Sex     39772 non-null  object
 2   Age     39772 non-null  int64 
 3   Height  39772 non-null  int64 
 4   Weight  39772 non-null  int64 
 5   Team    39772 non-null  object
 6   NOC     39772 non-null  object
 7   Games   39772 non-null  object
 8   Year    39772 non-null  int64 
 9   Season  39772 non-null  object
 10  City    39772 non-null  object
 11  Sport   39772 non-null  object
 12  Event   39772 non-null  object
 13  Medal   39772 non-null  object
 14  region  39772 non-null  object
dtypes: int64(4), object(11)
memory usage: 4.9+ MB


In [213]:
#number of different medals from 1896 to 2016
number_of_different_medals=df['Medal'].value_counts()
number_of_different_medals

Gold      13369
Bronze    13295
Silver    13108
Name: Medal, dtype: int64

In [182]:
df['Year'].nunique()

35

In [212]:
#number of Medals in each year
years_df = df.groupby('Year').count()['Medal']
years_df.sort_index()

Year
1896     143
1900     593
1904     486
1906     458
1908     831
1912     941
1920    1308
1924     962
1928     823
1932     739
1936    1025
1948     987
1952    1033
1956    1043
1960    1058
1964    1215
1968    1256
1972    1414
1976    1531
1980    1602
1984    1698
1988    1845
1992    2030
1994     331
1996    1842
1998     440
2000    2004
2002     478
2004    2001
2006     526
2008    2048
2010     520
2012    1941
2014     597
2016    2023
Name: Medal, dtype: int64

In [200]:
#number of women who have won 'Gold' medals
women_with_gold=df[(df['Sex'] == 'F') & (df['Medal'] == 'Gold')].count()['Medal']
women_with_gold

3747

In [201]:
#number of men who have won 'Gold' medals
men_with_gold=df[(df['Sex'] == 'M') & (df['Medal'] == 'Gold')].count()['Medal']
men_with_gold

9622